In [2]:
from xgboost import XGBRegressor

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 200)

import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'svg'

from tqdm import tqdm

from statsmodels.tsa.tsatools import lagmat

import matplotlib.pylab as plt
%matplotlib inline

In [3]:
transactions = pd.read_csv('transactions.csv')
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

# transactions['pos_amount'] = transactions.amount.apply(lambda x: 0 if x<0 else np.log(x + 1))
transactions['neg_amount'] = transactions.amount.apply(lambda x: 0 if x>0 else -x)

transactions.drop(['amount', 'term_id', 'tr_datetime'], 1, inplace=True)

test_transactions = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']


train_grid = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                          index=transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

for tr_table in tqdm([transactions, test_transactions, train_grid]):
    tr_table['week_num'] = (tr_table['day'] - 2) // 7
    tr_table['week_day'] = (tr_table['day'] - 2) % 7
    tr_table['month_num'] = tr_table['day'] // 28
    tr_table['month_day'] = tr_table['day'] % 28
    
merge_col_names = ['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code']

train_transactions = pd.merge(
    train_grid,
    transactions.groupby(merge_col_names)[['neg_amount']].sum().reset_index(),
    how='left').fillna(0)

train_transactions = pd.merge(
    train_transactions,
    transactions.groupby(merge_col_names)[['customer_id']].count().reset_index(),
    how='left').fillna(0).astype(np.int32)
train_transactions.columns = np.hstack([train_transactions.columns[:-1], ['n_transactions']])

train_transactions['log_neg_amount'] = train_transactions.neg_amount.apply(lambda x: np.log(x + 1))
# train_transactions['log_pos_amount'] = train_transactions.pos_amount.apply(lambda x: np.log(x + 1))
train_transactions.head()

100%|██████████| 3/3 [00:00<00:00,  2.00it/s]


,mcc_code,day,week_num,week_day,month_num,month_day,neg_amount,n_transactions,log_neg_amount
0,4814,0,-1,5,0,0,11098744,2365,16.222343
1,4814,1,-1,6,0,1,7881825,1697,15.880070
2,4814,2,0,0,0,2,6777480,1524,15.729116
3,4814,3,0,1,0,3,9277943,1937,16.043151
4,4814,4,0,2,0,4,9999757,1943,16.118071


In [24]:
train_shift = train_transactions.copy()
train_shift['week_num'] += 1
train_shift['prev_log_neg_amount'] = train_shift.log_neg_amount
train_shift['prev_n_trans'] = train_shift.n_transactions

train_shift = train_shift[['week_num', 'week_day', 'mcc_code', 'prev_log_neg_amount', 'prev_n_trans']]
train_transactions = pd.merge(
    train_transactions, train_shift, 
    on=['week_num', 'week_day', 'mcc_code'],
    how='left').fillna(0)
test_transactions = pd.merge(
    test_transactions, train_shift, 
    on=['week_num', 'week_day', 'mcc_code'],
    how='left').fillna(0)

train_transactions['log_neg_amount'] = \
    train_transactions.log_neg_amount - train_transactions.prev_log_neg_amount
train_transactions.n_transactions = train_transactions.n_transactions - train_transactions['prev_n_trans']
train_transactions.drop(['prev_log_neg_amount', 'prev_n_trans'], 1, inplace=True)
start_log_neg = train_transactions[train_transactions.day < 7][
    ['mcc_code', 'day', 'log_neg_amount', 'n_transactions']]
train_transactions.loc[train_transactions.day < 7, 'log_neg_amount'] = 0
train_transactions.loc[train_transactions.day < 7, 'n_transactions'] = 0
train_transactions.head(10)

,mcc_code,day,week_num,week_day,month_num,month_day,neg_amount,n_transactions,log_neg_amount
0,4814,0,-1,5,0,0,11098744,0.0,0.000000
1,4814,1,-1,6,0,1,7881825,0.0,0.000000
2,4814,2,0,0,0,2,6777480,0.0,0.000000
3,4814,3,0,1,0,3,9277943,0.0,0.000000
4,4814,4,0,2,0,4,9999757,0.0,0.000000
5,4814,5,0,3,0,5,9501177,0.0,0.000000
6,4814,6,0,4,0,6,9553618,0.0,0.000000
7,4814,7,0,5,0,7,9084047,-372.0,-0.200312
8,4814,8,0,6,0,8,7641198,-90.0,-0.031005
9,4814,10,1,1,0,10,10398283,52.0,0.114001


In [25]:
train = train_transactions.copy()

helper = train.copy().set_index(['mcc_code', 'week_num'])
helper['week_mean'] = train_transactions.groupby(['mcc_code', 'week_num']).mean()['log_neg_amount']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'week_num'])
helper['week_std'] = train_transactions.groupby(['mcc_code', 'week_num']).std()['log_neg_amount']
train = helper.reset_index().copy()
train.head()

helper = train.copy().set_index(['mcc_code', 'week_num'])
helper['week_mean_transactions'] = train_transactions.groupby(['mcc_code', 'week_num']).mean()['n_transactions']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'week_num'])
helper['week_std_transactions'] = train_transactions.groupby(['mcc_code', 'week_num']).std()['n_transactions']
train = helper.reset_index().copy()
train.head()

,mcc_code,week_num,day,week_day,month_num,month_day,neg_amount,n_transactions,log_neg_amount,week_mean,week_std,week_mean_transactions,week_std_transactions
0,4814,-1,0,5,0,0,11098744,0.0,0.0,0.000000,0.000000,0.0,0.000000
1,4814,-1,1,6,0,1,7881825,0.0,0.0,0.000000,0.000000,0.0,0.000000
2,4814,0,2,0,0,2,6777480,0.0,0.0,-0.033045,0.074657,-66.0,139.039563
3,4814,0,3,1,0,3,9277943,0.0,0.0,-0.033045,0.074657,-66.0,139.039563
4,4814,0,4,2,0,4,9999757,0.0,0.0,-0.033045,0.074657,-66.0,139.039563


In [26]:
helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_mean'] = train_transactions.groupby(['mcc_code', 'month_num']).mean()['log_neg_amount']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_std'] = train_transactions.groupby(['mcc_code', 'month_num']).std()['log_neg_amount']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_median'] = train_transactions.groupby(['mcc_code', 'month_num']).median()['log_neg_amount']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_min'] = train_transactions.groupby(['mcc_code', 'month_num']).min()['log_neg_amount']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_max'] = train_transactions.groupby(['mcc_code', 'month_num']).max()['log_neg_amount']
train = helper.reset_index().copy()

#n_transactions
helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_mean_transactions'] = train_transactions.groupby(['mcc_code', 'month_num']).mean()['n_transactions']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_std_transactions'] = train_transactions.groupby(['mcc_code', 'month_num']).std()['n_transactions']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_median_transactions'] = train_transactions.groupby(['mcc_code', 'month_num']).median()['n_transactions']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_min_transactions'] = train_transactions.groupby(['mcc_code', 'month_num']).min()['n_transactions']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_max_transactions'] = train_transactions.groupby(['mcc_code', 'month_num']).max()['n_transactions']
train = helper.reset_index().copy()

TIMESTAT_COLS = train.columns.difference(train_transactions.columns)

train.head()

,mcc_code,month_num,week_num,day,week_day,month_day,neg_amount,n_transactions,log_neg_amount,week_mean,week_std,week_mean_transactions,week_std_transactions,month_mean,month_std,month_median,month_min,month_max,month_mean_transactions,month_std_transactions,month_median_transactions,month_min_transactions,month_max_transactions
0,4814,0,-1,0,5,0,11098744,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.001733,0.069766,0.0,-0.200312,0.114001,8.035714,120.325941,0.0,-372.0,222.0
1,4814,0,-1,1,6,1,7881825,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.001733,0.069766,0.0,-0.200312,0.114001,8.035714,120.325941,0.0,-372.0,222.0
2,4814,0,0,2,0,2,6777480,0.0,0.0,-0.033045,0.074657,-66.0,139.039563,-0.001733,0.069766,0.0,-0.200312,0.114001,8.035714,120.325941,0.0,-372.0,222.0
3,4814,0,0,3,1,3,9277943,0.0,0.0,-0.033045,0.074657,-66.0,139.039563,-0.001733,0.069766,0.0,-0.200312,0.114001,8.035714,120.325941,0.0,-372.0,222.0
4,4814,0,0,4,2,4,9999757,0.0,0.0,-0.033045,0.074657,-66.0,139.039563,-0.001733,0.069766,0.0,-0.200312,0.114001,8.035714,120.325941,0.0,-372.0,222.0


In [27]:
week_lag_max = 35
# week_pos_lag_max = 20
for week_shift in tqdm(np.arange(1, week_lag_max)):
    train_shift = train.copy()
    train_shift['week_num'] += week_shift
    train_shift['prev_week_{}_neg'.format(week_shift)] = train_shift.log_neg_amount
    train_shift['prev_week_{}_mean_neg'.format(week_shift)] = train_shift.week_mean
    train_shift['prev_week_{}_std_neg'.format(week_shift)] = train_shift.week_std
    
    #n_transactions
    train_shift['prev_week_{}_n_trans'.format(week_shift)] = train_shift.n_transactions
    train_shift['prev_week_{}_mean_n_trans'.format(week_shift)] = train_shift.week_mean_transactions
    train_shift['prev_week_{}_std_n_trans'.format(week_shift)] = train_shift.week_std_transactions
    
    train_shift = train_shift[[
        'week_num', 'week_day', 'mcc_code',
        'prev_week_{}_neg'.format(week_shift),
        'prev_week_{}_mean_neg'.format(week_shift),
        'prev_week_{}_std_neg'.format(week_shift),
        #added
        'prev_week_{}_n_trans'.format(week_shift),
        'prev_week_{}_mean_n_trans'.format(week_shift),
        'prev_week_{}_std_n_trans'.format(week_shift),
    ]]
    train_transactions = pd.merge(
        train_transactions, train_shift, 
        on=['week_num', 'week_day', 'mcc_code'],
        how='left').fillna(0)
    test_transactions = pd.merge(
        test_transactions, train_shift, 
        on=['week_num', 'week_day', 'mcc_code'],
        how='left').fillna(0)
train_transactions.head()

100%|██████████| 34/34 [00:07<00:00,  3.20it/s]


,mcc_code,day,week_num,week_day,month_num,month_day,neg_amount,n_transactions,log_neg_amount,prev_week_1_neg,prev_week_1_mean_neg,prev_week_1_std_neg,prev_week_1_n_trans,prev_week_1_mean_n_trans,prev_week_1_std_n_trans,prev_week_2_neg,prev_week_2_mean_neg,prev_week_2_std_neg,prev_week_2_n_trans,prev_week_2_mean_n_trans,prev_week_2_std_n_trans,prev_week_3_neg,prev_week_3_mean_neg,prev_week_3_std_neg,prev_week_3_n_trans,prev_week_3_mean_n_trans,prev_week_3_std_n_trans,prev_week_4_neg,prev_week_4_mean_neg,prev_week_4_std_neg,prev_week_4_n_trans,prev_week_4_mean_n_trans,prev_week_4_std_n_trans,prev_week_5_neg,prev_week_5_mean_neg,prev_week_5_std_neg,prev_week_5_n_trans,prev_week_5_mean_n_trans,prev_week_5_std_n_trans,prev_week_6_neg,prev_week_6_mean_neg,prev_week_6_std_neg,prev_week_6_n_trans,prev_week_6_mean_n_trans,prev_week_6_std_n_trans,prev_week_7_neg,prev_week_7_mean_neg,prev_week_7_std_neg,prev_week_7_n_trans,prev_week_7_mean_n_trans,prev_week_7_std_n_trans,prev_week_8_neg,prev_week_8_mean_neg,prev_week_8_std_neg,prev_week_8_n_trans,prev_week_8_mean_n_trans,prev_week_8_std_n_trans,prev_week_9_neg,prev_week_9_mean_neg,prev_week_9_std_neg,prev_week_9_n_trans,prev_week_9_mean_n_trans,prev_week_9_std_n_trans,prev_week_10_neg,prev_week_10_mean_neg,prev_week_10_std_neg,prev_week_10_n_trans,prev_week_10_mean_n_trans,prev_week_10_std_n_trans,prev_week_11_neg,prev_week_11_mean_neg,prev_week_11_std_neg,prev_week_11_n_trans,prev_week_11_mean_n_trans,prev_week_11_std_n_trans,prev_week_12_neg,prev_week_12_mean_neg,prev_week_12_std_neg,prev_week_12_n_trans,prev_week_12_mean_n_trans,prev_week_12_std_n_trans,prev_week_13_neg,prev_week_13_mean_neg,prev_week_13_std_neg,prev_week_13_n_trans,prev_week_13_mean_n_trans,prev_week_13_std_n_trans,prev_week_14_neg,prev_week_14_mean_neg,prev_week_14_std_neg,prev_week_14_n_trans,prev_week_14_mean_n_trans,prev_week_14_std_n_trans,prev_week_15_neg,prev_week_15_mean_neg,prev_week_15_std_neg,prev_week_15_n_trans,prev_week_15_mean_n_trans,prev_week_15_std_n_trans,prev_week_16_neg,...,prev_week_18_std_neg,prev_week_18_n_trans,prev_week_18_mean_n_trans,prev_week_18_std_n_trans,prev_week_19_neg,prev_week_19_mean_neg,prev_week_19_std_neg,prev_week_19_n_trans,prev_week_19_mean_n_trans,prev_week_19_std_n_trans,prev_week_20_neg,prev_week_20_mean_neg,prev_week_20_std_neg,prev_week_20_n_trans,prev_week_20_mean_n_trans,prev_week_20_std_n_trans,prev_week_21_neg,prev_week_21_mean_neg,prev_week_21_std_neg,prev_week_21_n_trans,prev_week_21_mean_n_trans,prev_week_21_std_n_trans,prev_week_22_neg,prev_week_22_mean_neg,prev_week_22_std_neg,prev_week_22_n_trans,prev_week_22_mean_n_trans,prev_week_22_std_n_trans,prev_week_23_neg,prev_week_23_mean_neg,prev_week_23_std_neg,prev_week_23_n_trans,prev_week_23_mean_n_trans,prev_week_23_std_n_trans,prev_week_24_neg,prev_week_24_mean_neg,prev_week_24_std_neg,prev_week_24_n_trans,prev_week_24_mean_n_trans,prev_week_24_std_n_trans,prev_week_25_neg,prev_week_25_mean_neg,prev_week_25_std_neg,prev_week_25_n_trans,prev_week_25_mean_n_trans,prev_week_25_std_n_trans,prev_week_26_neg,prev_week_26_mean_neg,prev_week_26_std_neg,prev_week_26_n_trans,prev_week_26_mean_n_trans,prev_week_26_std_n_trans,prev_week_27_neg,prev_week_27_mean_neg,prev_week_27_std_neg,prev_week_27_n_trans,prev_week_27_mean_n_trans,prev_week_27_std_n_trans,prev_week_28_neg,prev_week_28_mean_neg,prev_week_28_std_neg,prev_week_28_n_trans,prev_week_28_mean_n_trans,prev_week_28_std_n_trans,prev_week_29_neg,prev_week_29_mean_neg,prev_week_29_std_neg,prev_week_29_n_trans,prev_week_29_mean_n_trans,prev_week_29_std_n_trans,prev_week_30_neg,prev_week_30_mean_neg,prev_week_30_std_neg,prev_week_30_n_trans,prev_week_30_mean_n_trans,prev_week_30_std_n_trans,prev_week_31_neg,prev_week_31_mean_neg,prev_week_31_std_neg,prev_week_31_n_trans,prev_week_31_mean_n_trans,prev_week_31_std_n_trans,prev_week_32_neg,prev_week_32_mean_neg,prev_week_32_std_neg,prev_week_32_n_trans,prev_week_32_mean_n_trans,prev_week_32_std_n_trans,prev_week_33_neg,pr

In [28]:
month_lag_max = 10
for month_shift in tqdm(np.arange(1, month_lag_max)):
    train_shift = train.copy()
    train_shift['month_num'] += month_shift
    train_shift['prev_month_{}_mean_neg'.format(month_shift)] = train_shift.month_mean
    train_shift['prev_month_{}_std_neg'.format(month_shift)] = train_shift.month_std
    train_shift['prev_month_{}_median_neg'.format(month_shift)] = train_shift.month_median
    train_shift['prev_month_{}_max_neg'.format(month_shift)] = train_shift.month_max
    train_shift['prev_month_{}_min_neg'.format(month_shift)] = train_shift.month_min
    
    #added
    train_shift['prev_month_{}_mean_n_trans'.format(month_shift)] = train_shift.month_mean_transactions
    train_shift['prev_month_{}_std_n_trans'.format(month_shift)] = train_shift.month_std_transactions
    train_shift['prev_month_{}_median_n_trans'.format(month_shift)] = train_shift.month_median_transactions
    train_shift['prev_month_{}_max_n_trans'.format(month_shift)] = train_shift.month_max_transactions
    train_shift['prev_month_{}_min_n_trans'.format(month_shift)] = train_shift.month_min_transactions
    
    train_shift = train_shift[[
        'month_num', 'day', 'mcc_code',
        'prev_month_{}_mean_neg'.format(month_shift),
        'prev_month_{}_std_neg'.format(month_shift),
        'prev_month_{}_median_neg'.format(month_shift),
        'prev_month_{}_max_neg'.format(month_shift),
        'prev_month_{}_min_neg'.format(month_shift),
        'prev_month_{}_mean_n_trans'.format(month_shift),
        'prev_month_{}_std_n_trans'.format(month_shift),
        'prev_month_{}_median_n_trans'.format(month_shift),
        'prev_month_{}_max_n_trans'.format(month_shift),
        'prev_month_{}_min_n_trans'.format(month_shift),
    ]]
    train_transactions = pd.merge(
        train_transactions, train_shift, 
        on=['month_num', 'day', 'mcc_code'],
        how='left').fillna(0)
    test_transactions = pd.merge(
        test_transactions, train_shift, 
        on=['month_num', 'day', 'mcc_code'],
        how='left').fillna(0)
train_transactions.head()

100%|██████████| 9/9 [00:03<00:00,  2.29it/s]


,mcc_code,day,week_num,week_day,month_num,month_day,neg_amount,n_transactions,log_neg_amount,prev_week_1_neg,prev_week_1_mean_neg,prev_week_1_std_neg,prev_week_1_n_trans,prev_week_1_mean_n_trans,prev_week_1_std_n_trans,prev_week_2_neg,prev_week_2_mean_neg,prev_week_2_std_neg,prev_week_2_n_trans,prev_week_2_mean_n_trans,prev_week_2_std_n_trans,prev_week_3_neg,prev_week_3_mean_neg,prev_week_3_std_neg,prev_week_3_n_trans,prev_week_3_mean_n_trans,prev_week_3_std_n_trans,prev_week_4_neg,prev_week_4_mean_neg,prev_week_4_std_neg,prev_week_4_n_trans,prev_week_4_mean_n_trans,prev_week_4_std_n_trans,prev_week_5_neg,prev_week_5_mean_neg,prev_week_5_std_neg,prev_week_5_n_trans,prev_week_5_mean_n_trans,prev_week_5_std_n_trans,prev_week_6_neg,prev_week_6_mean_neg,prev_week_6_std_neg,prev_week_6_n_trans,prev_week_6_mean_n_trans,prev_week_6_std_n_trans,prev_week_7_neg,prev_week_7_mean_neg,prev_week_7_std_neg,prev_week_7_n_trans,prev_week_7_mean_n_trans,prev_week_7_std_n_trans,prev_week_8_neg,prev_week_8_mean_neg,prev_week_8_std_neg,prev_week_8_n_trans,prev_week_8_mean_n_trans,prev_week_8_std_n_trans,prev_week_9_neg,prev_week_9_mean_neg,prev_week_9_std_neg,prev_week_9_n_trans,prev_week_9_mean_n_trans,prev_week_9_std_n_trans,prev_week_10_neg,prev_week_10_mean_neg,prev_week_10_std_neg,prev_week_10_n_trans,prev_week_10_mean_n_trans,prev_week_10_std_n_trans,prev_week_11_neg,prev_week_11_mean_neg,prev_week_11_std_neg,prev_week_11_n_trans,prev_week_11_mean_n_trans,prev_week_11_std_n_trans,prev_week_12_neg,prev_week_12_mean_neg,prev_week_12_std_neg,prev_week_12_n_trans,prev_week_12_mean_n_trans,prev_week_12_std_n_trans,prev_week_13_neg,prev_week_13_mean_neg,prev_week_13_std_neg,prev_week_13_n_trans,prev_week_13_mean_n_trans,prev_week_13_std_n_trans,prev_week_14_neg,prev_week_14_mean_neg,prev_week_14_std_neg,prev_week_14_n_trans,prev_week_14_mean_n_trans,prev_week_14_std_n_trans,prev_week_15_neg,prev_week_15_mean_neg,prev_week_15_std_neg,prev_week_15_n_trans,prev_week_15_mean_n_trans,prev_week_15_std_n_trans,prev_week_16_neg,...,prev_week_33_std_neg,prev_week_33_n_trans,prev_week_33_mean_n_trans,prev_week_33_std_n_trans,prev_week_34_neg,prev_week_34_mean_neg,prev_week_34_std_neg,prev_week_34_n_trans,prev_week_34_mean_n_trans,prev_week_34_std_n_trans,prev_month_1_mean_neg,prev_month_1_std_neg,prev_month_1_median_neg,prev_month_1_max_neg,prev_month_1_min_neg,prev_month_1_mean_n_trans,prev_month_1_std_n_trans,prev_month_1_median_n_trans,prev_month_1_max_n_trans,prev_month_1_min_n_trans,prev_month_2_mean_neg,prev_month_2_std_neg,prev_month_2_median_neg,prev_month_2_max_neg,prev_month_2_min_neg,prev_month_2_mean_n_trans,prev_month_2_std_n_trans,prev_month_2_median_n_trans,prev_month_2_max_n_trans,prev_month_2_min_n_trans,prev_month_3_mean_neg,prev_month_3_std_neg,prev_month_3_median_neg,prev_month_3_max_neg,prev_month_3_min_neg,prev_month_3_mean_n_trans,prev_month_3_std_n_trans,prev_month_3_median_n_trans,prev_month_3_max_n_trans,prev_month_3_min_n_trans,prev_month_4_mean_neg,prev_month_4_std_neg,prev_month_4_median_neg,prev_month_4_max_neg,prev_month_4_min_neg,prev_month_4_mean_n_trans,prev_month_4_std_n_trans,prev_month_4_median_n_trans,prev_month_4_max_n_trans,prev_month_4_min_n_trans,prev_month_5_mean_neg,prev_month_5_std_neg,prev_month_5_median_neg,prev_month_5_max_neg,prev_month_5_min_neg,prev_month_5_mean_n_trans,prev_month_5_std_n_trans,prev_month_5_median_n_trans,prev_month_5_max_n_trans,prev_month_5_min_n_trans,prev_month_6_mean_neg,prev_month_6_std_neg,prev_month_6_median_neg,prev_month_6_max_neg,prev_month_6_min_neg,prev_month_6_mean_n_trans,prev_month_6_std_n_trans,prev_month_6_median_n_trans,prev_month_6_max_n_trans,prev_month_6_min_n_trans,prev_month_7_mean_neg,prev_month_7_std_neg,prev_month_7_median_neg,prev_month_7_max_neg,prev_month_7_min_neg,prev_month_7_mean_n_trans,prev_month_7_std_n_trans,prev_month_7_median_n_trans,prev_month_7_max_n_trans,prev_month_7_min_n_trans,prev_month_8_mean_neg,prev_month_8_std_neg,prev_month_8_median_neg,pr

In [39]:
dummy_train = pd.get_dummies(train_transactions, columns=['mcc_code'])
dummy_test = pd.get_dummies(test_transactions, columns=['mcc_code'])

def rmsle(predicted, actual):
    assert(len(predicted) == len(actual))
    p = np.log(np.array(predicted) + 1)
    a = np.log(np.array(actual) + 1)
    return (((p - a)**2).sum() / len(predicted))**0.5

def rmsle_by_logs(predicted, actual):
    assert(len(predicted) == len(actual))
    return (((predicted - actual)**2).sum() / len(predicted))**0.5

def eval_model(labeled_data, target_col_name, clf, day_shifts=np.arange(90, 0, -15)):
    max_day = labeled_data.day.max()
    c = labeled_data.columns.difference([target_col_name])
    metric_by_shift = {}
    for day_shift in tqdm(day_shifts): 
        train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
        test_sample = labeled_data[labeled_data.day > max_day - day_shift]
        clf.fit(train_sample[c], train_sample[target_col_name])
        test_sample['predicted'] = clf.predict(test_sample[c])
        metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
            test_sample['predicted'], test_sample[target_col_name])
    return metric_by_shift, test_sample

def test_weeks_preparation(X_test):
    X_test_by_weeks = []
    current_test_week = X_test[X_test.week_num == 65].copy()
    X_test_by_weeks.append(current_test_week)
    removing_month_cols = ['prev_month_1_mean_neg', 'prev_month_1_std_neg', 'prev_month_1_median_neg', 
                           'prev_month_1_max_neg', 'prev_month_1_min_neg',
#                            'prev_month_1_mean_pos', 'prev_month_1_std_pos', 'prev_month_1_median_pos'
                          ]
    removing_week_cols = []
    for prev_week_index in range(1, 5):
        current_test_week = X_test[X_test.week_num == 65 + prev_week_index].copy()
        current_test_week.drop(removing_month_cols, 1, inplace=True)
        removing_week_cols += [
            'prev_week_{}_neg'.format(prev_week_index),
            'prev_week_{}_mean_neg'.format(prev_week_index),
            'prev_week_{}_std_neg'.format(prev_week_index),
            'prev_week_{}_n_trans'.format(prev_week_index),
            'prev_week_{}_mean_n_trans'.format(prev_week_index),
            'prev_week_{}_std_n_trans'.format(prev_week_index),
            
        ]
        current_test_week.drop(removing_week_cols, 1, inplace=True)
        X_test_by_weeks.append(current_test_week)
    return X_test_by_weeks
        
def get_agile_prediction(X_train, y_train, X_test, clf, test_form):
    X_test_by_weeks = test_weeks_preparation(X_test)
    out_form = pd.DataFrame()
    for prev_week_index in tqdm(range(5)):
        current_test = X_test_by_weeks[prev_week_index] 
        drop_cols = X_train.columns.difference(current_test.columns)
        current_train = X_train.drop(drop_cols, 1)
        clf.fit(current_train, y_train)
        predicted_log_volume = clf.predict(current_test)
        current_out_form = test_form[test_form.week_num == 65 + prev_week_index].copy()
        current_out_form['id'] = current_out_form[['mcc_code', 'day']].apply(
            lambda x: '-'.join(map(str, x)), axis=1)
        current_out_form['volume'] = np.expm1(predicted_log_volume)
        out_form = out_form.append(current_out_form[['id', 'volume']])
    return out_form

In [40]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.02, max_depth=6, n_estimators=350, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'n_transactions', 'neg_amount',]
    
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
metric_by_shift, predict = eval_model(
    dummy_train.drop(drop_cols, 1),
    'log_neg_amount', clf, [30])
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))


100%|██████████| 1/1 [01:42<00:00, 102.43s/it]

RMSLE with 30 days: 3.3725250779964533


In [36]:
predict

array([ 0.30016518,  0.25346154, -0.11691248, ..., -7.73465157,
        2.16995573,  1.35650015], dtype=float32)